# Dünya Üniversite Sıralamaları Analizi (Temel EDA)
Bu notebook, dünya üniversite sıralamalarını analiz etmek ve görselleştirmek amacıyla hazırlanmıştır. **İleri seviye makine öğrenmesi** yöntemlerine yer vermeden, verilerin temel temizliği ve keşifsel veri analizi üzerinde duracağız.

**Kullanılacak Veri Setleri (Örnek):**
- `timesData.csv` (Times Higher Education verileri)
- Gerektiğinde diğer CSV dosyaları

**Neler Yapacağız?**
1. Veriyi yükleyip genel bir bakış atmak (info, head, isnull vb.).
2. Veride gözlemlenen hataları/eksik değerleri temizlemek.
3. Temel istatistiksel özetleri ve görselleştirmeleri yapmak (bar chart, histogram, scatterplot vb.).
4. Değişkenler arasındaki korelasyonu (ısı haritası) incelemek.
5. Zaman serisi (varsa yıl bazında) trendleri keşfetmek.
6. Sonuçları özetlemek.

---

# 1. Gerekli Kütüphanelerin Yüklenmesi

In [1]:
# Gerekli kütüphaneleri yükleme
!pip install plotly --quiet

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import plot
import plotly.figure_factory as ff

pd.set_option("display.max_columns", None)


- `plotly` ve `pandas` kütüphaneleri temel veri işleme ve görselleştirme için yeterli olacak.
- `d.set_option`("display.max_columns", None) ile daha fazla sütunu görüntüleyebileceğiz.

# 2. Verinin Yüklenmesi ve İlk İnceleme
- Bu aşamada veride hangi sütunların olduğunu, tiplerinin ne olduğunu ve ne kadar eksik değer bulunduğunu görüyoruz.

In [2]:
# Örnek: timesData.csv yolunu belirt
df = pd.read_csv("/kaggle/input/world-university-rankings/timesData.csv")

# İlk beş satır
print("Veri setinin ilk 5 satırı:")
display(df.head())

# Veri seti hakkında özet bilgi
print("\nVeri setinin bilgi özeti:")
df.info()

# Eksik değer durumunu inceleme
print("\nEksik değer sayıları:")
print(df.isnull().sum())


Veri setinin ilk 5 satırı:


,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011



Veri setinin bilgi özeti:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2603 non-null   object 
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2603 non-null   object 
 8   total_score             2603 non-null   object 
 9   num_students            2544 non-null   object 
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   object 
 12  female_male_ratio       2370 non-null   object 
 13  year                    2603 non-null   int64  
dtypes: float64(4)

# 3. Veri Temizleme ve Sütun İsimlerini Düzenleme
- Eksik değerleri ortalama veya medyan ile doldurduk.
- `dunya_siralamasi` içinde “=13” veya “1-2” gibi ifadelere özel temizlik işlemi uyguladık.
- Kadın-erkek oranlarını iki ayrı sütuna böldük.

In [3]:
# Sütun adlarını Türkçeye çevirme
df.rename(columns={
    'world_rank': 'dunya_siralamasi',
    'university_name': 'universite_adi',
    'country': 'ulke',
    'teaching': 'egitim',
    'international': 'uluslararasi',
    'research': 'arastirma',
    'citations': 'atiflar',
    'income': 'gelir',
    'total_score': 'toplam_puan',
    'num_students': 'ogrenci_sayisi',
    'student_staff_ratio': 'ogrenci_personel_orani',
    'international_students': 'uluslararasi_ogrenci_orani',
    'female_male_ratio': 'kadin_erkek_orani',
    'year': 'yil'
}, inplace=True)

# "-" karakterini NaN ile değiştirelim
df.replace("-", np.nan, inplace=True)

# Dünya sıralaması içindeki özel durumları düzeltme
def temizlenmis_ranking(deger):
    if isinstance(deger, str):
        if "=" in deger:
            return int(deger.replace("=", "").strip())
        elif "-" in deger:
            bas, son = map(int, deger.split("-"))
            return (bas + son) / 2
        else:
            return int(deger)
    return deger

df["dunya_siralamasi"] = df["dunya_siralamasi"].apply(temizlenmis_ranking)

# Ülke isimlerinde bariz yazım hatalarını düzelt
duzeltme_dict = {
    "Unisted States of America": "United States of America",
    "Unted Kingdom": "United Kingdom"
}
df["ulke"] = df["ulke"].replace(duzeltme_dict)

# Sayısal sütunları float'a çevir, eksikleri ortalama ile doldur
for col in ["egitim", "uluslararasi", "arastirma", "atiflar", "gelir", "toplam_puan"]:
    df[col] = df[col].astype(float)
    df[col] = df[col].fillna(df[col].mean())

# Öğrenci sayısı
df["ogrenci_sayisi"] = df["ogrenci_sayisi"].replace(",", "", regex=True).astype(float)
df["ogrenci_sayisi"] = df["ogrenci_sayisi"].fillna(df["ogrenci_sayisi"].median())

# Öğrenci-personel oranı
df["ogrenci_personel_orani"] = df["ogrenci_personel_orani"].astype(float)
df["ogrenci_personel_orani"] = df["ogrenci_personel_orani"].fillna(df["ogrenci_personel_orani"].mean())

# Uluslararası öğrenci oranı (string'den % işaretini silerek float'a)
df["uluslararasi_ogrenci_orani"] = df["uluslararasi_ogrenci_orani"].str.replace("%", "").astype(float)
df["uluslararasi_ogrenci_orani"] = df["uluslararasi_ogrenci_orani"].fillna(df["uluslararasi_ogrenci_orani"].mean())

# Kadın-Erkek oranı
df["kadin_erkek_orani"] = df["kadin_erkek_orani"].replace({"-": "50 : 50", np.nan: "50 : 50"})

# Kadın ve erkek sütunlarına ayırma
df["kadin"], df["erkek"] = zip(*df["kadin_erkek_orani"].str.split(" : ").apply(lambda x: (float(x[0]), float(x[1]))))
df.drop(columns=["kadin_erkek_orani"], inplace=True)

print("Temizlenmiş veri seti ilk 5 satır:")
display(df.head())

print("\nEksik değerler (temizlik sonrası):")
print(df.isnull().sum())


Temizlenmiş veri seti ilk 5 satır:


,dunya_siralamasi,universite_adi,ulke,egitim,uluslararasi,arastirma,atiflar,gelir,toplam_puan,ogrenci_sayisi,ogrenci_personel_orani,uluslararasi_ogrenci_orani,yil,kadin,erkek
0,1.0,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.500000,96.1,20152.0,8.9,25.0,2011,50.0,50.0
1,2.0,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.700000,96.0,2243.0,6.9,27.0,2011,33.0,67.0
2,3.0,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.500000,95.6,11074.0,9.0,33.0,2011,37.0,63.0
3,4.0,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.300000,94.3,15596.0,7.8,22.0,2011,42.0,58.0
4,5.0,Princeton University,United States of America,90.9,70.3,95.4,99.9,48.979874,94.2,7929.0,8.4,27.0,2011,45.0,55.0



Eksik değerler (temizlik sonrası):
dunya_siralamasi              0
universite_adi                0
ulke                          0
egitim                        0
uluslararasi                  0
arastirma                     0
atiflar                       0
gelir                         0
toplam_puan                   0
ogrenci_sayisi                0
ogrenci_personel_orani        0
uluslararasi_ogrenci_orani    0
yil                           0
kadin                         0
erkek                         0
dtype: int64


# 4. Keşifsel Veri Analizi (EDA)

1. **Temel İstatistiksel Özeti Gösterme**  
   - `describe()` gibi fonksiyonlar kullanarak sayısal sütunların (ör. `egitim`, `arastirma`, `ogrenci_sayisi`) minimum, maksimum, ortalama ve çeyreklik değerlerini görüntüleyebiliriz.  
   - Bu sayede veri setinde sıra dışı değerler (outlier) veya genel dağılım eğilimleri hakkında ilk fikirleri ediniriz.

2. **Ülkelerin Üniversite Sayıları**  
   - Veri setindeki üniversitelerin hangi ülkelere ait olduğunu sayar, hangi ülkelerin en çok üniversiteye sahip olduğunu görürüz.  
   - Bar grafiği (ör. `px.bar`) ile ülkeler bazında hızlı bir karşılaştırma yapabiliriz.

3. **Öğrenci Sayısı Dağılımı**  
   - Histogram (ör. `px.histogram`) kullanarak üniversitelerin öğrenci sayılarını dağılım biçiminde görselleştirebiliriz.  
   - Öğrenci sayısı çok yüksek veya çok düşük olan üniversiteler hangi grupta yoğunlaşıyor, rahatça gözlemleriz.

4. **Kadın-Erkek Oranlarının Dağılımı**  
   - Veri setinde `kadin` ve `erkek` sütunlarına ayrılmış oranlar bulunuyorsa, bu oranların dağılımını bir box plot (ör. `px.box`) ile görebiliriz.  
   - Böylece kadın ve erkek öğrenci oranlarının tipik değerleri, uç değerleri ve aralıkları hakkında fikir sahibi oluruz.


In [4]:
# describe() metodu ile sayısal sütunların temel istatistiklerini görelim
df_describe = df.describe()
df_describe

,dunya_siralamasi,egitim,uluslararasi,arastirma,atiflar,gelir,toplam_puan,ogrenci_sayisi,ogrenci_personel_orani,uluslararasi_ogrenci_orani,yil,kadin,erkek
count,2603.000000,2603.000000,2603.000000,2603.000000,2603.000000,2603.000000,2603.000000,2603.000000,2603.000000,2603.000000,2603.000000,2603.000000,2603.000000
mean,253.930657,37.801498,52.007440,35.910257,60.921629,48.979874,59.846128,23805.244333,18.445283,15.444006,2014.075682,49.712639,50.287361
std,186.185879,17.604218,22.065565,21.254805,23.073219,20.273287,8.694885,17480.190768,11.328041,10.454213,1.685733,10.355834,10.355834
min,1.000000,9.900000,7.100000,2.900000,1.200000,24.200000,41.400000,462.000000,0.600000,0.000000,2011.000000,1.000000,0.000000
25%,109.000000,24.700000,33.500000,19.600000,45.500000,33.600000,57.100000,12800.000000,12.050000,8.000000,2013.000000,47.000000,44.000000
50%,213.000000,33.900000,50.400000,30.500000,62.500000,42.600000,59.846128,20851.000000,16.300000,14.000000,2014.000000,51.000000,49.000000
75%,325.500000,46.400000,69.000000,47.250000,79.050000,56.000000,59.846128,29787.000000,21.200000,21.000000,2016.000000,56.000000,53.000000
max,700.500000,99.700000,100.000000,99.400000,100.000000,100.000000,96.100000,379231.000000,162.600000,82.000000,2016.000000,100.000000,99.000000


In [5]:
ulke_sayilari = df["ulke"].value_counts()

fig1 = px.bar(
    x=ulke_sayilari.index,
    y=ulke_sayilari.values,
    labels={"x": "Ülke", "y": "Üniversite Sayısı"},
    title="Ülkelerin Üniversite Sayıları"
)
fig1.show()


In [6]:
fig2 = px.histogram(
    df,
    x="ogrenci_sayisi",
    nbins=50,
    labels={"ogrenci_sayisi": "Öğrenci Sayısı"},
    title="Öğrenci Sayısının Dağılımı"
)
fig2.show()


In [7]:
fig3 = px.box(
    df,
    y=["kadin", "erkek"],
    labels={"value": "Oran (%)", "variable": "Cinsiyet"},
    title="Kadın ve Erkek Oranlarının Box Plot Dağılımı"
)
fig3.show()


# 5.İki Değişkenli Analiz
- Scatter plot’lar, iki değişken arasındaki ilişkileri görselleştirmek için kullanılabilir.
- `color="ulke"` ile farklı ülkeleri farklı renklerle görebiliriz.

In [8]:
# Uluslararası Puan vs Toplam Puan
fig4 = px.scatter(
    df,
    x="uluslararasi",
    y="toplam_puan",
    color="ulke",
    labels={"uluslararasi": "Uluslararası Puan", "toplam_puan": "Toplam Puan"},
    title="Uluslararası Puan ve Toplam Puan İlişkisi"
)
fig4.show()

# Dünya Sıralaması vs Toplam Puan
fig5 = px.scatter(
    df,
    x="dunya_siralamasi",
    y="toplam_puan",
    color="ulke",
    labels={"dunya_siralamasi": "Dünya Sıralaması", "toplam_puan": "Toplam Puan"},
    title="Dünya Sıralaması ve Toplam Puan İlişkisi"
)
fig5.show()

# Eğitim vs Araştırma
fig6 = px.scatter(
    df,
    x="egitim",
    y="arastirma",
    color="ulke",
    title="Eğitim ve Araştırma Puanı İlişkisi"
)
fig6.show()

# Gelir vs Toplam Puan
fig7 = px.scatter(
    df,
    x="gelir",
    y="toplam_puan",
    color="ulke",
    title="Gelir ve Toplam Puan İlişkisi"
)
fig7.show()


# 6.Korelasyon Analizi (Isı Haritası)
- Isı haritası, değişkenlerin birbirleriyle (lineer) korelasyonuna hızlı bir bakış sağlar.
- Örneğin `arastirma` (araştırma puanı) ile `atiflar` (atıf puanı) arasında yüksek korelasyon beklenebilir.

In [9]:
import plotly.figure_factory as ff

num_cols = [
    "egitim", "uluslararasi", "arastirma", "atiflar", "gelir", 
    "toplam_puan", "ogrenci_sayisi", "ogrenci_personel_orani", 
    "uluslararasi_ogrenci_orani", "kadin", "erkek"
]

# Korelasyon matrisini hesapla ve değerleri yuvarla
corr_matrix = df[num_cols].corr().round(2)

# Z, X, Y değerleri
z = corr_matrix.values
x = corr_matrix.columns.tolist()
y = corr_matrix.columns.tolist()

# Annotated Heatmap oluştururken annotation_text'e matris değerlerini ver
fig = ff.create_annotated_heatmap(
    z=z,
    x=x,
    y=y,
    colorscale="Viridis",
    showscale=True,
    annotation_text=z.astype(str)  # string'e çevirerek etikete yazdır
)

# Anotasyonların (matris içi yazılar) font boyutunu döngüyle ayarla
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i].font.size = 10  # istediğin boyut

# X eksenindeki etiketleri döndürerek daha kolay okunur hale getirebilirsin
fig.update_xaxes(tickangle=-45)

# Başlık ekle
fig.update_layout(title="Değişkenler Arası Korelasyon Isı Haritası")

fig.show()


# 7. Zaman Serisi Analizi (Yıl Bazında)
- 2011’den 2016’ya kadar ortalama puan değişimini görebilirsiniz

In [10]:
# Yıllara göre ortalama toplam puan
yearly_mean = df.groupby("yil")["toplam_puan"].mean().reset_index()

fig9 = px.line(
    yearly_mean,
    x="yil",
    y="toplam_puan",
    title="Yıllara Göre Ortalama Toplam Puan Değişimi"
)
fig9.show()


# 8. Sonuçlar ve Özet
1. **Ülke Dağılımları**  
   - En fazla üniversiteye sahip ülkeler arasında ABD ve Birleşik Krallık öne çıkıyor.

2. **Değişkenler Arası İlişkiler**  
   - `arastirma` ve `atiflar` arasında beklediğimiz gibi yüksek bir korelasyon var.  
   - `toplam_puan`, araştırma ve atıf puanlarından büyük ölçüde etkileniyor gibi görünüyor.

3. **Zaman Serisi**  
   - Yıllara göre ortalama puanlar incelendiğinde (örnek: 2011-2016), belli eğilimler veya değişiklikler gözlemlenebilir.  
   - Bazı yıllarda genel puan ortalaması artarken bazı yıllarda düşüş kaydedilebilir.

4. **Genel Değerlendirme**  
   - Dünyanın en prestijli üniversiteleri genellikle yüksek araştırma, yüksek atıf puanına sahip; aynı zamanda bu üniversiteler ülkelerin ekonomik ve akademik alt yapısıyla da ilişkili.  
   - İleri analizler (ör. daha detaylı istatistiksel testler, farklı veri setleriyle birleştirme, harita üzerinde görselleştirme) yapılarak konuyu derinleştirebilirsiniz.